In [1]:
import drawSvg as draw
import numpy as np

C:\Anaconda3\lib\site-packages\drawSvg\raster.py:18: RuntimeWarning: Failed to import CairoSVG. drawSvg will be unable to output PNG or other raster image formats. See https://github.com/cduck/drawSvg#prerequisites for more details.
Original OSError: no library called "cairo-2" was found
no library called "cairo" was found
cannot load library 'C:\GTK\bin\libcairo-2.dll': error 0xc1
cannot load library 'libcairo.so.2': error 0x7e
cannot load library 'libcairo.2.dylib': error 0x7e
cannot load library 'libcairo-2.dll': error 0xc1
  warnings.warn(msg, RuntimeWarning)


In [2]:
#helper functions
def normal_length1(vector, side):
    unit_vector = vector/np.linalg.norm(vector)
    if side == 0: #positive
        unit_normal = np.array([unit_vector[1], -1*unit_vector[0]])
    elif side == 1: #negative side
        unit_normal = np.array([-1*unit_vector[1], unit_vector[0]])
    return unit_normal

#formula for calculating cosA between to vectors
def calc_degree(vector1, vector2):
    cos_alpha = np.dot(vector1, vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))
    degree = np.arccos(cos_alpha)*180/np.pi
    return degree

In [3]:
line_color = 'blue'
offset_len = 10
text_size = 8 #text size questionable, color too

def lin_domain(x0, y0, x1, y1, side=0, name=""):
    #draw line from 5' to 3'
    d.append(draw.Line(x0, y0, x1, y1, stroke = line_color))
    
    #label has to be on its correct side
    if name != "":
        #unit vector
        vector = np.array([x1 - x0, y1 - y0])
        label_offset = normal_length1(vector, side)*offset_len
        x_text, y_text = np.array([x0,y0])+ vector/2 + label_offset
        
        d.append(draw.Text(name, text_size, x_text, y_text, fill = line_color))

In [36]:
#rn default distance between domains 20 ? paired_dist = 20
loop_color = 'red'
offset_len = 10
text_size = 8
  
def hairpin_loop(x0, y0, x1, y1, side=0, name=""):
    #calculate Arc arguments
    r = np.sqrt((x1-x0)**2 + (y1-y0)**2) #rn radius is the distance between paired regions, will change

    #vector is pointing from point 0 -> point 1
    vector = np.array([x1 - x0, y1 - y0])
    vec_len = np.sqrt(vector[0]**2 + vector[1]**2)
    unit_normal = normal_length1(vector, side)
    
    #calculate center and label anchor with r and vector
    diag = np.sqrt(r**2 - (vec_len/2)**2)
    cx, cy = np.array([x0,y0])+ vector/2 + diag*unit_normal
    x_text, y_text = np.array([x0,y0])+ vector/2 + (diag+r+offset_len)*unit_normal
       
    vec_center_0 = np.array([x0-cx,y0-cy])
    vec_center_1 = np.array([x1-cx,y1-cy])
    
    #bit unclear about start/end degrees, but works
    if side == 0:
        startdeg = calc_degree(vec_center_0, np.array([1,0]))*np.sign(vec_center_0[1])
        enddeg = calc_degree(vec_center_1, np.array([1,0]))*np.sign(vec_center_1[1])
    elif side == 1:
        enddeg = calc_degree(vec_center_0, np.array([1,0]))*np.sign(vec_center_0[1])
        startdeg = calc_degree(vec_center_1, np.array([1,0]))*np.sign(vec_center_1[1])
    
    #draw
    d.append(draw.Arc(cx, cy, r, startdeg, enddeg,
            stroke=loop_color, stroke_width=0.9, fill="none")) #stroke_width will be also important
    
    #if without name, no label is added
    if name != "":
        d.append(draw.Text(name, text_size, x_text, y_text, fill=loop_color)) #label

In [5]:
def unpaired_loop():
    pass

In [6]:
def multiloop():
    pass

In [80]:
ORIGIN = (-20,-10)
SCALE_METRIC = 2
CANVAS_SIZE = [200, 150]

In [81]:
#image instancing
d = draw.Drawing(CANVAS_SIZE[0], CANVAS_SIZE[1], origin=ORIGIN, displayInline=False)
d.setPixelScale(SCALE_METRIC)

#domains but will be for loop
lin_domain(70, 100, 20, 50, side=0, name="a")
hairpin_loop(20, 50, 30, 40, side=0, name="b")
lin_domain(30, 40, 80, 90,  side=0, name="a*")

#print but probably can do differently
d

In [82]:
#the image
d = draw.Drawing(CANVAS_SIZE[0], CANVAS_SIZE[1], origin=ORIGIN, displayInline=False)
d.setPixelScale(SCALE_METRIC)

lin_domain(0, 50, 100, 50, name="name", side=1)
hairpin_loop(100, 50, 100, 30, name="name", side=1)
lin_domain(100, 30, 0, 30, name="name", side=1)

d

In [76]:
#given start coords(5'), angle, side and domain length - draw a whole hairpin from it
basepair_distance = 15

def calculate_coords_and_draw_hairpin(x_start, y_start, angle, dom_len, side):
    #if angle is between 1 and 179 like here, y can be assumed to be 1
    angle_rad = angle/180*np.pi
    
    vec0_1 = np.array([np.cos(angle_rad), np.sin(angle_rad)])
    x1, y1 = np.array([x_start, y_start]) + vec0_1*dom_len
    
    vec1_2 = normal_length1(vector=vec0_1, side=side)
    x2, y2 = np.array([x1, y1]) + vec1_2*basepair_distance
    
    vec2_3 = normal_length1(vector=vec1_2, side=side)
    x3, y3 = np.array([x2, y2]) + vec2_3*dom_len
    
    lin_domain(x_start, y_start, x1, y1, name="a", side=0) #side is bugged here rn
    hairpin_loop(x1, y1, x2, y2, name="b", side=0)
    lin_domain(x2, y2, x3, y3, name="a*", side=0)

In [83]:
d = draw.Drawing(CANVAS_SIZE[0], CANVAS_SIZE[1], origin=ORIGIN, displayInline=False)
d.setPixelScale(SCALE_METRIC)

calculate_coords_and_draw_hairpin(x_start=30, y_start=30, angle=110, dom_len=60, side=1)

d